In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!nvidia-smi

Wed Jul 14 18:11:39 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.42.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   62C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
! pip install keras==2.3.1

In [ ]:
# import tensorflow.keras as keras
import numpy as np
import os
import random

! pip install keras-tuner
import tensorflow as tf
import numpy as np
import os
import random
from tensorflow.keras.applications import VGG19
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, Dense, Dropout, Input, Concatenate, GlobalMaxPool1D, LSTM, MaxPool1D
from tensorflow.keras.layers import ReLU, BatchNormalization, Bidirectional, concatenate, Reshape, TimeDistributed
from tensorflow.keras.layers import Conv2D, ConvLSTM2D, MaxPool2D, SpatialDropout2D, Softmax, ELU, Layer, Masking
from tensorflow.keras.layers import GlobalAveragePooling2D, GlobalMaxPool2D, LeakyReLU, LSTMCell, RNN, InputSpec
from tensorflow.keras.layers import concatenate, Conv1DTranspose, Conv2DTranspose, Flatten, StackedRNNCells
from tensorflow.keras.initializers import Constant
from tensorflow.keras.models import Model
from tensorflow.keras import Input
from tensorflow import keras as keras
from tensorflow.keras.optimizers import Adam, SGD, RMSprop
from matplotlib import pyplot as plt
import kerastuner as kt
from kerastuner.tuners import RandomSearch, BayesianOptimization, Hyperband
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import GlobalAveragePooling2D, GlobalAveragePooling1D, Permute, multiply, Activation
import tensorflow.keras.backend as K

from tensorflow.keras import backend as K
from tensorflow.keras import activations
from tensorflow.keras import initializers
from tensorflow.keras import regularizers
from tensorflow.keras import constraints


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:24: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.


In [ ]:
def load_X(path):
    X_signals = []
    files = os.listdir(path)
    files.sort(key=str.lower)
    #['train_acc_x.txt', 'train_acc_y.txt', 'train_acc_z.txt', 'train_gyr_x.txt', 'train_gyr_y.txt', 'train_gyr_z.txt']
    for my_file in files:
        fileName = os.path.join(path,my_file)
        file = open(fileName, 'r')
        X_signals.append(
            [np.array(cell, dtype=np.float32) for cell in [
                row.strip().split(' ') for row in file
            ]]
        )
        file.close()
        #X_signals = 6*totalStepNum*128
    X_signals = np.transpose(np.array(X_signals), (1, 0, 2))#(totalStepNum*6*128)
    return X_signals.reshape(-1,6,128,1)#(totalStepNum*6*128*1)

def load_y(y_path):
    file = open(y_path, 'r')
    # Read dataset from disk, dealing with text file's syntax
    y_ = np.array(
        [elem for elem in [
            row.replace('  ', ' ').strip().split(' ') for row in file
        ]],
        dtype=np.int32
    )
    file.close()
    # Substract 1 to each output class for friendly 0-based indexing
    y_ = y_ - 1
    #one_hot
    y_ = y_.reshape(len(y_))
    n_values = int(np.max(y_)) + 1
    return np.eye(n_values)[np.array(y_, dtype=np.int32)]  # Returns FLOATS


def weight_variable(shape):
    initial = tf.random.truncated_normal(shape, std_dev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)


X_train = load_X('/content/drive/Shareddrives/MMH/TIFS20/Dataset #2/train/Inertial Signals')
X_test = load_X('/content/drive/Shareddrives/MMH/TIFS20/Dataset #2/test/Inertial Signals')
train_label = load_y('/content/drive/Shareddrives/MMH/TIFS20/Dataset #2/train/y_train.txt')
test_label = load_y('/content/drive/Shareddrives/MMH/TIFS20/Dataset #2/test/y_test.txt')
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], X_train.shape[2]))
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], X_test.shape[2]))

batch_size = 512
num_classes = 20

In [ ]:
X_train.shape, train_label.shape, X_test.shape, test_label.shape

((44339, 6, 128), (44339, 20), (4936, 6, 128), (4936, 20))

In [ ]:
def tuned_lstm_model(hp):
    # model = Sequential()
    input_seq = Input(shape=(6, 128))
    processed = LSTM(hp.Int('units', min_value=32, max_value=512, step=32), input_shape=(6, 128))(input_seq)
    processed = Dropout(0.5)(processed)
    # x = (Dense(118*8, activation='relu'))(processed)
    # x = (Dense(118*4, activation='relu'))(x)
    processed =ReLU()(processed)
    activated = (Dense(20, activation='softmax'))(processed)
    model = Model(inputs=input_seq, outputs=activated)
    model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-1, 1e-2, 1e-3, 3*1e-1, 3*1e-2, 3*1e-3, 3*1e-4, 1e-4, 1.0])), \
                  loss='categorical_crossentropy',metrics=['accuracy'])
    return model

In [ ]:
tuner = RandomSearch(
    tuned_lstm_model,
    objective='val_accuracy',
    max_trials=10,
    directory='output2',
    project_name='TIFS20')

tuner.search(X_train, train_label, epochs=200, validation_split=0.1, shuffle=True, callbacks=[EarlyStopping(monitor='val_loss', patience=30)])

Trial 10 Complete [00h 16m 46s]
val_accuracy: 0.7652232646942139

Best val_accuracy So Far: 0.9781236052513123
Total elapsed time: 01h 09m 39s
INFO:tensorflow:Oracle triggered exit


In [ ]:
model = tuner.get_best_models(num_models=1)[0]
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 6, 128)]          0         
_________________________________________________________________
lstm (LSTM)                  (None, 384)               787968    
_________________________________________________________________
dropout (Dropout)            (None, 384)               0         
_________________________________________________________________
re_lu (ReLU)                 (None, 384)               0         
_________________________________________________________________
dense (Dense)                (None, 20)                7700      
Total params: 795,668
Trainable params: 795,668
Non-trainable params: 0
_________________________________________________________________


In [ ]:
history = model.fit(X_train, train_label, validation_split = 0.1, epochs=100, shuffle=True, batch_size=512, callbacks=[EarlyStopping('val_loss', patience=30)])#, callbacks=[es_callback])

Epoch 1/100
78/78 [==============================] - 2s 14ms/step - loss: 0.0315 - accuracy: 0.9904 - val_loss: 0.1086 - val_accuracy: 0.9786
Epoch 2/100
78/78 [==============================] - 1s 8ms/step - loss: 0.0169 - accuracy: 0.9945 - val_loss: 0.1062 - val_accuracy: 0.9799
Epoch 3/100
78/78 [==============================] - 1s 8ms/step - loss: 0.0123 - accuracy: 0.9960 - val_loss: 0.1051 - val_accuracy: 0.9797
Epoch 4/100
78/78 [==============================] - 1s 8ms/step - loss: 0.0093 - accuracy: 0.9971 - val_loss: 0.1037 - val_accuracy: 0.9793
Epoch 5/100
78/78 [==============================] - 1s 8ms/step - loss: 0.0087 - accuracy: 0.9971 - val_loss: 0.1034 - val_accuracy: 0.9790
Epoch 6/100
78/78 [==============================] - 1s 8ms/step - loss: 0.0082 - accuracy: 0.9975 - val_loss: 0.1059 - val_accuracy: 0.9795
Epoch 7/100
78/78 [==============================] - 1s 8ms/step - loss: 0.0061 - accuracy: 0.9984 - val_loss: 0.1044 - val_accuracy: 0.9799
Epoch 8/100


In [ ]:
import matplotlib.pyplot as plt
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

In [ ]:
score = model.evaluate(X_test, test_label, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 0.5563589930534363
Test accuracy: 0.9252431392669678


In [ ]:
model.save('/content/drive/Shareddrives/MMH/models/tuned_lstm_94.57_dataset2.h5')

In [ ]:
# Without Dropout:
# Test loss: 0.3413163721561432
# Test accuracy: 0.945704996585846